In [2]:
# KB차차차 중고차 매물 크롤링 - 페이지별 carSeq 추출
import requests
import json
import pandas as pd
from datetime import datetime
import time
from bs4 import BeautifulSoup
import re ,difflib
from urllib.parse import urlsplit, urlunsplit

BASE_URL = "https://www.kbchachacha.com"

session = requests.Session()
session.headers.update({
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36',
    'Accept': '*/*',
    'Accept-Language': 'ko,ko-KR;q=0.9,en-US;q=0.8,en;q=0.7',
    'Referer': 'https://www.kbchachacha.com/public/search/main.kbc',
    'Host': 'www.kbchachacha.com'
})

def get_car_seqs_from_page(page_num):
    url = f"https://www.kbchachacha.com/public/search/list.empty?page={page_num}&sort=-orderDate"
    
    try:
        res = session.get(url, timeout=10)
        if res.status_code == 200:
            soup = BeautifulSoup(res.text, 'html.parser')
            
            # .area 클래스에서 data-car-seq 추출
            areas = soup.select('.area')
            page_car_seqs = []
            
            for area in areas:
                car_seq = area.get('data-car-seq')
                if car_seq:
                    page_car_seqs.append(car_seq)
            
            print(f"페이지 {page_num}: {len(page_car_seqs)}개 carSeq 발견")
            return page_car_seqs
        else:
            print(f"페이지 {page_num}: HTTP {res.status_code}")
            return []
    except Exception as e:
        print(f"페이지 {page_num} 오류: {e}")
        return []

def crawl_multiple_pages(max_pages=5, delay=1):
    """여러 페이지를 크롤링하여 모든 carSeq를 수집합니다."""
    all_car_seqs = []
    
    print(f"KB차차차 {max_pages}페이지 크롤링 시작...")
    
    for page in range(1, max_pages + 1):
        print(f"\n페이지 {page} 처리 중...")
        
        page_car_seqs = get_car_seqs_from_page(page)
        
        if page_car_seqs:
            all_car_seqs.extend(page_car_seqs)
            print(f"페이지 {page} carSeq: {page_car_seqs[:5]}...")  # 처음 5개만 출력
        else:
            print(f"페이지 {page}: carSeq 없음")
            break  # 더 이상 데이터가 없으면 중단
        
        # 서버 부하 방지
        time.sleep(delay)
    
    # 중복 제거
    unique_car_seqs = list(set(all_car_seqs))
    
    print(f"\n=== 크롤링 완료 ===")
    print(f"총 수집된 carSeq: {len(all_car_seqs)}개")
    print(f"중복 제거 후: {len(unique_car_seqs)}개")
    print(f"carSeq 목록: {unique_car_seqs[:10]}...")  # 처음 10개만 출력
    
    return unique_car_seqs

# 크롤링 실행 (테스트용으로 3페이지)
print("KB차차차 페이지별 carSeq 크롤링 시작...")
all_car_seqs = crawl_multiple_pages(max_pages=3, delay=2)

KB차차차 페이지별 carSeq 크롤링 시작...
KB차차차 3페이지 크롤링 시작...

페이지 1 처리 중...
페이지 1: 40개 carSeq 발견
페이지 1 carSeq: ['27500410', '27483929', '27464096', '27464001', '27483491']...

페이지 2 처리 중...
페이지 2: 40개 carSeq 발견
페이지 2 carSeq: ['27377699', '27355166', '27350872', '27285384', '27214013']...

페이지 3 처리 중...
페이지 3: 40개 carSeq 발견
페이지 3 carSeq: ['27492138', '27479879', '27463512', '26806339', '26766361']...

=== 크롤링 완료 ===
총 수집된 carSeq: 120개
중복 제거 후: 120개
carSeq 목록: ['27500410', '27443098', '27085349', '26987698', '27459029', '27359644', '26970934', '27159076', '27160183', '27377699']...


In [3]:
def get_car_info_via_api(car_seqs):
    """carSeq 리스트를 API로 보내서 일괄로 차량 정보를 받아옵니다."""
    
    # API 엔드포인트
    api_url = "https://www.kbchachacha.com/public/car/common/recent/car/list.json"
    
    # carSeq를 쉼표로 구분된 문자열로 변환
    car_seq_str = ",".join(car_seqs)
    
    # POST 요청 데이터
    payload = {
        'gotoPage': 1,
        'pageSize': len(car_seqs),
        'carSeqVal': car_seq_str
    }
    
    print(f"\n=== API 요청 정보 ===")
    print(f"URL: {api_url}")
    print(f"carSeq 개수: {len(car_seqs)}")
    print(f"carSeqVal: {car_seq_str}")
    
    try:
        # POST 요청
        response = session.post(api_url, data=payload, timeout=10)
        
        print(f"HTTP 상태코드: {response.status_code}")
        print(f"응답 크기: {len(response.text)} bytes")
        
        if response.status_code == 200:
            # JSON 파싱
            data = response.json()
            
            print(f"✅ API 응답 성공!")
            print(f"총 차량 수: {data.get('totalCount', 0)}")
            print(f"실제 리스트 수: {len(data.get('list', []))}")
            
            return data.get('list', [])
        else:
            print(f"❌ HTTP 오류: {response.status_code}")
            return []
            
    except Exception as e:
        print(f"❌ API 요청 실패: {e}")
        return []

# 테스트: API를 통한 차량 정보 수집 (샘플 carSeq 사용)
test_car_seqs = ['27490092', '27483488', '27471263']  # 테스트용 carSeq

print(f"\n{'='*60}")
print("🚀 API를 통한 효율적인 차량 정보 수집 테스트")
print(f"{'='*60}")

car_info_list = get_car_info_via_api(test_car_seqs)

if car_info_list:
    print(f"\n=== 수집된 차량 정보 (상세) ===")
    for i, car in enumerate(car_info_list, 1):
        print(f"\n{i}. {car.get('makerName', '')} {car.get('carName', '')} {car.get('modelName', '')}")
        print(f"   carSeq: {car.get('carSeq')}")
        
        # 가격 정보 (priceMin, priceMax, priceRateMin, priceRateMax, priceRateAvg)
        sell_amt = car.get('sellAmt', 0)
        if sell_amt:
            print(f"   priceMin: {sell_amt}")
            print(f"   priceMax: {sell_amt}")
            print(f"   priceRateAvg: {sell_amt}")
        
        # 연식 (yymm)
        yymm = car.get('yymm', '')
        if yymm:
            print(f"   yymm: {yymm}")
        
        # 등록일 (regiDay)
        regi_day = car.get('regiDay', '')
        if regi_day:
            print(f"   regiDay: {regi_day}")
        
        # 제조사명 (makerName)
        maker_name = car.get('makerName', '')
        if maker_name:
            print(f"   makerName: {maker_name}")
        
        # 차량 사진 (photoName1)
        photo_name = car.get('photoName1', '')
        if photo_name:
            print(f"   photoName1: {photo_name}")
        
        # 기존 정보들
        print(f"   주행거리: {car.get('km', 0):,}km")
        print(f"   지역: {car.get('cityName', '')}")
        print(f"   차량번호: {car.get('carNo', '')}")
        print(f"   연료: {car.get('gasName', '')}")
        print(f"   사고이력: {car.get('accidentNo', '')}")
        print(f"   담보: {car.get('distraintInfo', 0)}")
        print(f"   저당: {car.get('mortgageInfo', 0)}")
        print(f"   세금체납: {car.get('taxDefaultInfo', 0)}")
        
        # 추가 API 필드들
        print(f"   배기량: {car.get('displacement', '')}")
        print(f"   변속기: {car.get('transmission', '')}")
        print(f"   색상: {car.get('color', '')}")
        print(f"   연식월: {car.get('yymmMonth', '')}")
        print(f"   등록월: {car.get('regiMonth', '')}")
        print(f"   판매상태: {car.get('sellStatus', '')}")
        print(f"   특이사항: {car.get('specialNote', '')}")
        print(f"   옵션: {car.get('optionList', '')}")
        print(f"   보증: {car.get('warranty', '')}")
        print(f"   할부: {car.get('installment', '')}")
        print(f"   리스: {car.get('lease', '')}")
        
        # API에서 사용 가능한 모든 필드 출력 (디버깅용)
        if i == 1:  # 첫 번째 차량만 전체 필드 출력
            print(f"\n   📋 API에서 제공하는 모든 필드:")
            for key, value in car.items():
                if value and value != '' and value != 0:
                    print(f"      {key}: {value}")
else:
    print("❌ API를 통한 정보 수집에 실패했습니다.")



🚀 API를 통한 효율적인 차량 정보 수집 테스트

=== API 요청 정보 ===
URL: https://www.kbchachacha.com/public/car/common/recent/car/list.json
carSeq 개수: 3
carSeqVal: 27490092,27483488,27471263
HTTP 상태코드: 200
응답 크기: 10051 bytes
✅ API 응답 성공!
총 차량 수: 3
실제 리스트 수: 3

=== 수집된 차량 정보 (상세) ===

1. 기아 더 뉴 기아 레이 1.0 가솔린
   carSeq: 27490092
   priceMin: 1630
   priceMax: 1630
   priceRateAvg: 1630
   yymm: 2023
   regiDay: 20231006
   makerName: 기아
   photoName1: 27490092_4656596854618635.jpeg
   주행거리: 19,707km
   지역: 경기
   차량번호: 116소1868
   연료: 
   사고이력: 
   담보: 0
   저당: 0
   세금체납: 0
   배기량: 
   변속기: 
   색상: 
   연식월: 
   등록월: 
   판매상태: 
   특이사항: 
   옵션: 
   보증: 
   할부: 
   리스: 

   📋 API에서 제공하는 모든 필드:
      carSeq: 27490092
      carNo: 116소1868
      makerCode: 102
      makerName: 기아
      classCode: 1236
      className: 레이
      carCode: 3306
      carName: 더 뉴 기아 레이
      modelCode: 26757
      modelName: 1.0 가솔린
      gradeCode: 002
      gradeName: 프레스티지
      regiDay: 20231006
      yymm: 2023
      km: 19707


In [4]:
# 🚀 제조사 정보를 통한 국산/수입 구분 시스템
def get_maker_info():
    """제조사 정보를 가져와서 makerCode와 countryCode 매핑을 생성합니다."""
    
    maker_url = "https://www.kbchachacha.com/public/search/carMaker.json?page=1&sort=-orderDate"
    
    try:
        response = requests.get(maker_url, timeout=10)
        
        if response.status_code == 200:
            data = response.json()
            maker_info = {}
            
            # 국산차 정보
            for maker in data.get('result', {}).get('국산', []):
                maker_info[maker['makerCode']] = {
                    'makerName': maker['makerName'],
                    'countryCode': maker['countryCode'],
                    'count': maker['count']
                }
            
            # 수입차 정보
            for maker in data.get('result', {}).get('수입', []):
                maker_info[maker['makerCode']] = {
                    'makerName': maker['makerName'],
                    'countryCode': maker['countryCode'],
                    'count': maker['count']
                }
            
            print(f"✅ 제조사 정보 수집 완료: 총 {len(maker_info)}개 제조사")
            return maker_info
        else:
            print(f"❌ 제조사 정보 수집 실패: HTTP {response.status_code}")
            return {}
            
    except Exception as e:
        print(f"❌ 제조사 정보 수집 오류: {e}")
        return {}

# 제조사 정보 수집
maker_info = get_maker_info()

# 제조사 정보 출력 (처음 10개만)
if maker_info:
    print(f"\n📋 제조사 정보 샘플:")
    for i, (code, info) in enumerate(list(maker_info.items())[:10]):
        print(f"   {code}: {info['makerName']} ({info['countryCode']}) - {info['count']}대")


✅ 제조사 정보 수집 완료: 총 82개 제조사

📋 제조사 정보 샘플:
   101: 현대 (국산) - 49364대
   189: 제네시스 (국산) - 7382대
   102: 기아 (국산) - 41947대
   103: 한국GM (국산) - 9293대
   105: 르노코리아 (국산) - 7727대
   104: KG모빌리티 (국산) - 7248대
   106: 기타 (국산) - 178대
   108: 벤츠 (수입) - 8418대
   107: BMW (수입) - 8367대
   109: 아우디 (수입) - 2803대


In [5]:
# 🚀 간단한 API 테스트 - session 문제 완전 해결
import requests

def get_car_info_simple(car_seqs):
    """간단한 API 호출 함수"""
    
    # API 엔드포인트
    api_url = "https://www.kbchachacha.com/public/car/common/recent/car/list.json"
    
    # carSeq를 쉼표로 구분된 문자열로 변환
    car_seq_str = ",".join(car_seqs)
    
    # POST 요청 데이터
    payload = {
        'gotoPage': 1,
        'pageSize': len(car_seqs),
        'carSeqVal': car_seq_str
    }
    
    print(f"\n=== API 요청 정보 ===")
    print(f"URL: {api_url}")
    print(f"carSeq 개수: {len(car_seqs)}")
    print(f"carSeqVal: {car_seq_str}")
    
    try:
        # session 생성
        session = requests.Session()
        session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36',
            'Accept': '*/*',
            'Accept-Language': 'ko,ko-KR;q=0.9,en-US;q=0.8,en;q=0.7',
            'Referer': 'https://www.kbchachacha.com/public/search/main.kbc',
            'Host': 'www.kbchachacha.com'
        })
        
        # POST 요청
        response = session.post(api_url, data=payload, timeout=10)
        
        print(f"HTTP 상태코드: {response.status_code}")
        print(f"응답 크기: {len(response.text)} bytes")
        
        if response.status_code == 200:
            # JSON 파싱
            data = response.json()
            
            print(f"✅ API 응답 성공!")
            print(f"총 차량 수: {data.get('totalCount', 0)}")
            print(f"실제 리스트 수: {len(data.get('list', []))}")
            
            return data.get('list', [])
        else:
            print(f"❌ HTTP 오류: {response.status_code}")
            return []
            
    except Exception as e:
        print(f"❌ API 요청 실패: {e}")
        return []

# 테스트 실행
test_car_seqs = ['27490092', '27483488', '27471263']

print(f"\n{'='*60}")
print("🚀 간단한 API 테스트")
print(f"{'='*60}")

car_list = get_car_info_simple(test_car_seqs)

# 결과 출력
if car_list:
    print(f"\n=== 수집된 차량 정보 ===")
    for i, car in enumerate(car_list, 1):
        print(f"\n{i}. {car.get('makerName', '')} {car.get('carName', '')} {car.get('modelName', '')}")
        print(f"   carSeq: {car.get('carSeq')}")
        
        # 요청하신 필드들
        sell_amt = car.get('sellAmt', 0)
        if sell_amt:
            print(f"   priceMin: {sell_amt}")
            print(f"   priceMax: {sell_amt}")
            print(f"   priceRateAvg: {sell_amt}")
        
        yymm = car.get('yymm', '')
        if yymm:
            print(f"   yymm: {yymm}")
        
        regi_day = car.get('regiDay', '')
        if regi_day:
            print(f"   regiDay: {regi_day}")
        
        maker_name = car.get('makerName', '')
        if maker_name:
            print(f"   makerName: {maker_name}")
        
        photo_name = car.get('photoName1', '')
        if photo_name:
            print(f"   photoName1: {photo_name}")
        
        # 기존 정보들
        print(f"   주행거리: {car.get('km', 0):,}km")
        print(f"   지역: {car.get('cityName', '')}")
        print(f"   차량번호: {car.get('carNo', '')}")
        print(f"   연료: {car.get('gasName', '')}")
        
        # 첫 번째 차량만 전체 필드 출력
        if i == 1:
            print(f"\n   📋 API에서 제공하는 모든 필드:")
            for key, value in car.items():
                if value and value != '' and value != 0:
                    print(f"      {key}: {value}")
else:
    print("❌ 차량 정보 수집에 실패했습니다.")



🚀 간단한 API 테스트

=== API 요청 정보 ===
URL: https://www.kbchachacha.com/public/car/common/recent/car/list.json
carSeq 개수: 3
carSeqVal: 27490092,27483488,27471263
HTTP 상태코드: 200
응답 크기: 10051 bytes
✅ API 응답 성공!
총 차량 수: 3
실제 리스트 수: 3

=== 수집된 차량 정보 ===

1. 기아 더 뉴 기아 레이 1.0 가솔린
   carSeq: 27490092
   priceMin: 1630
   priceMax: 1630
   priceRateAvg: 1630
   yymm: 2023
   regiDay: 20231006
   makerName: 기아
   photoName1: 27490092_4656596854618635.jpeg
   주행거리: 19,707km
   지역: 경기
   차량번호: 116소1868
   연료: 

   📋 API에서 제공하는 모든 필드:
      carSeq: 27490092
      carNo: 116소1868
      makerCode: 102
      makerName: 기아
      classCode: 1236
      className: 레이
      carCode: 3306
      carName: 더 뉴 기아 레이
      modelCode: 26757
      modelName: 1.0 가솔린
      gradeCode: 002
      gradeName: 프레스티지
      regiDay: 20231006
      yymm: 2023
      km: 19707
      sellAmtGbn: 109100
      sellAmt: 1630
      ownerYn: Y
      cityName: 경기
      photoName1: 27490092_4656596854618635.jpeg
      wishYn: N
      co

In [6]:
def get_maker_info():
    """제조사 정보를 가져와서 국산/수입 구분을 위한 매핑을 생성합니다."""
    
    maker_url = "https://www.kbchachacha.com/public/search/carMaker.json?page=1&sort=-orderDate"
    
    try:
        response = requests.get(maker_url, timeout=10)
        
        if response.status_code == 200:
            data = response.json()
            maker_info = {}
            
            # 국산차 정보
            for maker in data.get('result', {}).get('국산', []):
                maker_info[maker['makerCode']] = {
                    'makerName': maker['makerName'],
                    'countryCode': maker['countryCode'],
                    'count': maker['count']
                }
            
            # 수입차 정보
            for maker in data.get('result', {}).get('수입', []):
                maker_info[maker['makerCode']] = {
                    'makerName': maker['makerName'],
                    'countryCode': maker['countryCode'],
                    'count': maker['count']
                }
            
            print(f"✅ 제조사 정보 수집 완료: 총 {len(maker_info)}개 제조사")
            return maker_info
        else:
            print(f"❌ 제조사 정보 수집 실패: HTTP {response.status_code}")
            return {}
            
    except Exception as e:
        print(f"❌ 제조사 정보 수집 오류: {e}")
        return {}

# 제조사 정보 수집
print("📋 제조사 정보 수집 중...")
maker_info = get_maker_info()

# 테스트 실행
test_car_seqs = ['27490092', '27483488', '27471263']

print(f"\n{'='*60}")
print("🚀 완전한 KB차차차 크롤링 시스템")
print(f"{'='*60}")

car_list = get_car_info_simple(test_car_seqs)

# 결과 출력
if car_list:
    print(f"\n=== 수집된 차량 정보 (완전 버전) ===")
    for i, car in enumerate(car_list, 1):
        print(f"\n{i}. {car.get('makerName', '')} {car.get('carName', '')} {car.get('modelName', '')}")
        print(f"   carSeq: {car.get('carSeq')}")
        
        # 요청하신 필드들
        sell_amt = car.get('sellAmt', 0)
        if sell_amt:
            print(f"   priceMin: {sell_amt}")
            print(f"   priceMax: {sell_amt}")
            print(f"   priceRateAvg: {sell_amt}")
        
        yymm = car.get('yymm', '')
        if yymm:
            print(f"   yymm: {yymm}")
        
        regi_day = car.get('regiDay', '')
        if regi_day:
            print(f"   regiDay: {regi_day}")
        
        maker_name = car.get('makerName', '')
        if maker_name:
            print(f"   makerName: {maker_name}")
        
        photo_name = car.get('photoName1', '')
        if photo_name:
            print(f"   photoName1: {photo_name}")
        
        # 국산/수입 구분 (maker_info가 있는 경우)
        if maker_info:
            maker_code = car.get('makerCode', '')
            if maker_code in maker_info:
                country_code = maker_info[maker_code]['countryCode']
                print(f"   countryCode: {country_code}")
        
        # 기존 정보들
        print(f"   주행거리: {car.get('km', 0):,}km")
        print(f"   지역: {car.get('cityName', '')}")
        print(f"   차량번호: {car.get('carNo', '')}")
        print(f"   연료: {car.get('gasName', '')}")
        print(f"   사고이력: {car.get('accidentNo', '')}")
        print(f"   담보: {car.get('distraintInfo', 0)}")
        print(f"   저당: {car.get('mortgageInfo', 0)}")
        print(f"   세금체납: {car.get('taxDefaultInfo', 0)}")
        
        # 추가 정보들
        print(f"   배기량: {car.get('displacement', '')}")
        print(f"   변속기: {car.get('transmission', '')}")
        print(f"   색상: {car.get('color', '')}")
        print(f"   연식월: {car.get('yymmMonth', '')}")
        print(f"   등록월: {car.get('regiMonth', '')}")
        print(f"   판매상태: {car.get('sellStatus', '')}")
        print(f"   특이사항: {car.get('specialNote', '')}")
        print(f"   옵션: {car.get('optionList', '')}")
        print(f"   보증: {car.get('warranty', '')}")
        print(f"   할부: {car.get('installment', '')}")
        print(f"   리스: {car.get('lease', '')}")
        
        # 첫 번째 차량만 전체 필드 출력
        if i == 1:
            print(f"\n   📋 API에서 제공하는 모든 필드:")
            for key, value in car.items():
                if value and value != '' and value != 0:
                    print(f"      {key}: {value}")
else:
    print("❌ 차량 정보 수집에 실패했습니다.")


📋 제조사 정보 수집 중...
✅ 제조사 정보 수집 완료: 총 82개 제조사

🚀 완전한 KB차차차 크롤링 시스템

=== API 요청 정보 ===
URL: https://www.kbchachacha.com/public/car/common/recent/car/list.json
carSeq 개수: 3
carSeqVal: 27490092,27483488,27471263
HTTP 상태코드: 200
응답 크기: 10051 bytes
✅ API 응답 성공!
총 차량 수: 3
실제 리스트 수: 3

=== 수집된 차량 정보 (완전 버전) ===

1. 기아 더 뉴 기아 레이 1.0 가솔린
   carSeq: 27490092
   priceMin: 1630
   priceMax: 1630
   priceRateAvg: 1630
   yymm: 2023
   regiDay: 20231006
   makerName: 기아
   photoName1: 27490092_4656596854618635.jpeg
   countryCode: 국산
   주행거리: 19,707km
   지역: 경기
   차량번호: 116소1868
   연료: 
   사고이력: 
   담보: 0
   저당: 0
   세금체납: 0
   배기량: 
   변속기: 
   색상: 
   연식월: 
   등록월: 
   판매상태: 
   특이사항: 
   옵션: 
   보증: 
   할부: 
   리스: 

   📋 API에서 제공하는 모든 필드:
      carSeq: 27490092
      carNo: 116소1868
      makerCode: 102
      makerName: 기아
      classCode: 1236
      className: 레이
      carCode: 3306
      carName: 더 뉴 기아 레이
      modelCode: 26757
      modelName: 1.0 가솔린
      gradeCode: 002
      gradeName: 프레스티지


In [7]:
# 🚀 향상된 API 정보 수집 - 국산/수입 구분 포함
def get_car_info_via_api_enhanced(car_seqs, maker_info_dict):
    """carSeq 리스트를 API로 보내서 일괄로 차량 정보를 받아옵니다. (국산/수입 구분 포함)"""
    
    # API 엔드포인트
    api_url = "https://www.kbchachacha.com/public/car/common/recent/car/list.json"
    
    # carSeq를 쉼표로 구분된 문자열로 변환
    car_seq_str = ",".join(car_seqs)
    
    # POST 요청 데이터
    payload = {
        'gotoPage': 1,
        'pageSize': len(car_seqs),
        'carSeqVal': car_seq_str
    }
    
    print(f"\n=== API 요청 정보 ===")
    print(f"URL: {api_url}")
    print(f"carSeq 개수: {len(car_seqs)}")
    print(f"carSeqVal: {car_seq_str}")
    
    try:

        # POST 요청
        response = session.post(api_url, data=payload, timeout=10)
        
        print(f"HTTP 상태코드: {response.status_code}")
        print(f"응답 크기: {len(response.text)} bytes")
        
        if response.status_code == 200:
            # JSON 파싱
            data = response.json()
            
            print(f"✅ API 응답 성공!")
            print(f"총 차량 수: {data.get('totalCount', 0)}")
            print(f"실제 리스트 수: {len(data.get('list', []))}")
            
            # 차량 정보에 국산/수입 정보 추가
            car_list = data.get('list', [])
            for car in car_list:
                maker_code = car.get('makerCode', '')
                if maker_code in maker_info_dict:
                    car['countryCode'] = maker_info_dict[maker_code]['countryCode']
                    car['makerName_kor'] = maker_info_dict[maker_code]['makerName']
                else:
                    car['countryCode'] = '알수없음'
                    car['makerName_kor'] = car.get('makerName', '')
            
            return car_list
        else:
            print(f"❌ HTTP 오류: {response.status_code}")
            return []
            
    except Exception as e:
        print(f"❌ API 요청 실패: {e}")
        return []

# 테스트: 향상된 API를 통한 차량 정보 수집
test_car_seqs = ['27490092', '27483488', '27471263']  # 테스트용 carSeq

print(f"\n{'='*60}")
print("🚀 향상된 API를 통한 차량 정보 수집 테스트 (국산/수입 구분)")
print(f"{'='*60}")

# maker_info가 정의되지 않은 경우 빈 딕셔너리 사용
if 'maker_info' not in globals():
    print("⚠️ maker_info가 정의되지 않았습니다. 빈 딕셔너리를 사용합니다.")
    maker_info = {}

car_info_list = get_car_info_via_api_enhanced(test_car_seqs, maker_info)

if car_info_list:
    print(f"\n=== 수집된 차량 정보 (국산/수입 구분 포함) ===")
    for i, car in enumerate(car_info_list, 1):
        print(f"\n{i}. {car.get('makerName', '')} {car.get('carName', '')} {car.get('modelName', '')}")
        print(f"   carSeq: {car.get('carSeq')}")
        
        # 국산/수입 구분
        country_code = car.get('countryCode', '알수없음')
        print(f"   countryCode: {country_code}")
        
        # 가격 정보 (priceMin, priceMax, priceRateMin, priceRateMax, priceRateAvg)
        sell_amt = car.get('sellAmt', 0)
        if sell_amt:
            print(f"   priceMin: {sell_amt}")
            print(f"   priceMax: {sell_amt}")
            print(f"   priceRateAvg: {sell_amt}")
        
        # 연식 (yymm)
        yymm = car.get('yymm', '')
        if yymm:
            print(f"   yymm: {yymm}")
        
        # 등록일 (regiDay)
        regi_day = car.get('regiDay', '')
        if regi_day:
            print(f"   regiDay: {regi_day}")
        
        # 제조사명 (makerName)
        maker_name = car.get('makerName', '')
        if maker_name:
            print(f"   makerName: {maker_name}")
        
        # 차량 사진 (photoName1)
        photo_name = car.get('photoName1', '')
        if photo_name:
            print(f"   photoName1: {photo_name}")
        
        # 기존 정보들
        print(f"   주행거리: {car.get('km', 0):,}km")
        print(f"   지역: {car.get('cityName', '')}")
        print(f"   차량번호: {car.get('carNo', '')}")
        print(f"   연료: {car.get('gasName', '')}")
        print(f"   사고이력: {car.get('accidentNo', '')}")
        print(f"   담보: {car.get('distraintInfo', 0)}")
        print(f"   저당: {car.get('mortgageInfo', 0)}")
        print(f"   세금체납: {car.get('taxDefaultInfo', 0)}")
        
        # 추가 API 필드들
        print(f"   배기량: {car.get('displacement', '')}")
        print(f"   변속기: {car.get('transmission', '')}")
        print(f"   색상: {car.get('color', '')}")
        print(f"   연식월: {car.get('yymmMonth', '')}")
        print(f"   등록월: {car.get('regiMonth', '')}")
        print(f"   판매상태: {car.get('sellStatus', '')}")
        print(f"   특이사항: {car.get('specialNote', '')}")
        print(f"   옵션: {car.get('optionList', '')}")
        print(f"   보증: {car.get('warranty', '')}")
        print(f"   할부: {car.get('installment', '')}")
        print(f"   리스: {car.get('lease', '')}")
        
        # API에서 사용 가능한 모든 필드 출력 (디버깅용)
        if i == 1:  # 첫 번째 차량만 전체 필드 출력
            print(f"\n   📋 API에서 제공하는 모든 필드:")
            for key, value in car.items():
                if value and value != '' and value != 0:
                    print(f"      {key}: {value}")
else:
    print("❌ API를 통한 정보 수집에 실패했습니다.")



🚀 향상된 API를 통한 차량 정보 수집 테스트 (국산/수입 구분)

=== API 요청 정보 ===
URL: https://www.kbchachacha.com/public/car/common/recent/car/list.json
carSeq 개수: 3
carSeqVal: 27490092,27483488,27471263
HTTP 상태코드: 200
응답 크기: 10051 bytes
✅ API 응답 성공!
총 차량 수: 3
실제 리스트 수: 3

=== 수집된 차량 정보 (국산/수입 구분 포함) ===

1. 기아 더 뉴 기아 레이 1.0 가솔린
   carSeq: 27490092
   countryCode: 국산
   priceMin: 1630
   priceMax: 1630
   priceRateAvg: 1630
   yymm: 2023
   regiDay: 20231006
   makerName: 기아
   photoName1: 27490092_4656596854618635.jpeg
   주행거리: 19,707km
   지역: 경기
   차량번호: 116소1868
   연료: 
   사고이력: 
   담보: 0
   저당: 0
   세금체납: 0
   배기량: 
   변속기: 
   색상: 
   연식월: 
   등록월: 
   판매상태: 
   특이사항: 
   옵션: 
   보증: 
   할부: 
   리스: 

   📋 API에서 제공하는 모든 필드:
      carSeq: 27490092
      carNo: 116소1868
      makerCode: 102
      makerName: 기아
      classCode: 1236
      className: 레이
      carCode: 3306
      carName: 더 뉴 기아 레이
      modelCode: 26757
      modelName: 1.0 가솔린
      gradeCode: 002
      gradeName: 프레스티지
      regiDay: 2023

In [ ]:

from urllib.parse import urlsplit, urlunsplit

DETAIL = 'https://www.kbchachacha.com/public/car/detail.kbc'
BASE_HEADERS = {'User-Agent':'Mozilla/5.0','Accept-Language':'ko-KR,ko;q=0.9'}

def only_digits(s): 
    if not s: return None
    n = re.sub(r'[^0-9]', '', s)
    return int(n) if n else None

def clean_img(u):
    p = urlsplit(u)
    return urlunsplit((p.scheme, p.netloc, p.path, '', ''))

def parse_yymm(txt):
    # 예: '11년10월(11년형)' → '2011-10'
    m = re.search(r'(\d{2})년\s*(\d{1,2})월', txt or '')
    if not m: return None
    yy, mm = int(m.group(1)), int(m.group(2))
    year = 2000 + yy if yy < 70 else 1900 + yy
    return f'{year:04d}-{mm:02d}'

def fetch_car(car_seq: str) -> dict:
    s = requests.Session()
    html = s.get(DETAIL, params={'carSeq': car_seq}, headers=BASE_HEADERS, timeout=15).text
    soup = BeautifulSoup(html, 'html.parser')

    # 상단 요약
    name = (soup.select_one('.car-buy-name') or {}).get_text('\n', strip=True)
    price_txt = (soup.select_one('.car-buy-price strong') or {}).get_text(strip=True)
    sell_amt = only_digits(price_txt)

    top_spans = [x.get_text(strip=True) for x in soup.select('.car-buy-share .txt-info span')]
    yymm_txt = top_spans[0] if len(top_spans) > 0 else None
    km_txt   = top_spans[1] if len(top_spans) > 1 else None
    fuel     = top_spans[2] if len(top_spans) > 2 else None
    region   = top_spans[3] if len(top_spans) > 3 else None

    # 기본정보 표
    kv = {}
    for tr in soup.select('.detail-info-table tbody tr'):
        tds = tr.select('th,td')
        for i in range(0, len(tds), 2):
            k = tds[i].get_text(strip=True)
            v = tds[i+1].get_text(strip=True) if i+1 < len(tds) else ''
            kv[k] = v

    # 이미지
    imgs = [clean_img(img['src']) for img in soup.select('#btnCarPhotoView img[src]')]

    # 옵션 코드(선택)
    car_option = soup.select_one('input#carOption')
    option_codes = (car_option['value'].split(',') if car_option and car_option.has_attr('value') else [])

    # carHistorySeq(선택)
    m = re.search(r'carHistorySeq\s*=\s*"(\d+)"', html)
    car_history_seq = m.group(1) if m else None

    data = {
        'carSeq': car_seq,
        'title': name,
        'sellAmt': sell_amt,
        'yymm': parse_yymm(yymm_txt or kv.get('연식')),
        'km': only_digits(km_txt or kv.get('주행거리')),
        'fuel': (fuel or kv.get('연료')),
        'transmission': kv.get('변속기'),
        'class': kv.get('차종'),
        'displacementCc': only_digits(kv.get('배기량')),
        'color': kv.get('색상'),
        'region': region,
        'plateNo': kv.get('차량정보'),
        'ownerChangeCnt': only_digits((soup.select_one('.detail-info02 dl dd:nth-of-type(4)') or {}).get_text(strip=True)) or None,
        'imageUrls': imgs,
        'optionCodes': option_codes,
        'carHistorySeq': car_history_seq,
    }
    return data

# 사용
print(fetch_car('27470105'))

{'carSeq': '27470105', 'title': '(26거6975)한국GM 쉐보레 크루즈\n1.8 LTZ+ 기본형', 'sellAmt': 26697518, 'yymm': '2011-10', 'km': 131607, 'fuel': '가솔린', 'transmission': '오토', 'class': '준중형', 'displacementCc': 1796, 'color': '흰색', 'region': '인천', 'plateNo': '26거6975', 'ownerChangeCnt': 1, 'imageUrls': ['https://img.kbchachacha.com/IMG/carimg/l/img07/img2747/27470105_4126882981866427.jpg', 'https://img.kbchachacha.com/IMG/carimg/l/img07/img2747/27470105_4126883446632938.jpg', 'https://img.kbchachacha.com/IMG/carimg/l/img07/img2747/27470105_4126883326988752.jpg', 'https://img.kbchachacha.com/IMG/carimg/l/img07/img2747/27470105_4126883097808856.jpg', 'https://img.kbchachacha.com/IMG/carimg/l/img07/img2747/27470105_4126882661823483.jpg', 'https://img.kbchachacha.com/IMG/carimg/l/img07/img2747/27470105_4126883557606766.jpg', 'https://img.kbchachacha.com/IMG/carimg/l/img07/img2747/27470105_4126882791744151.jpg', 'https://img.kbchachacha.com/IMG/carimg/l/img07/img2747/27470105_4126882345704212.jpg', 'https

In [13]:
detail = 'https://www.kbchachacha.com/public/car/detail.kbc'
layer  = 'https://www.kbchachacha.com/public/layer/car/option/list.kbc'
car_seq = '27470105'

s = requests.Session()
base = {
    'User-Agent': 'Mozilla/5.0',
    'Accept-Language': 'ko-KR,ko;q=0.9',
}

# 1) 상세 페이지로 세션/쿠키 확보
s.get(detail, params={'carSeq': car_seq}, headers=base, timeout=15)

# 2) 레이어 POST (폼 전송 + AJAX/리퍼러 헤더)
headers = {
    **base,
    'X-Requested-With': 'XMLHttpRequest',
    'Referer': f'{detail}?carSeq={car_seq}',
    'Content-Type': 'application/x-www-form-urlencoded',
    'Accept': 'text/html, */*;q=0.1',
}
res = s.post(layer, data={'carSeq': car_seq}, headers=headers, timeout=15)
print(res.status_code)

# 3) HTML 파싱(예: 옵션 li 태그)
soup = BeautifulSoup(res.text, 'html.parser')

soup

200



<!DOCTYPE html>

<html lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="qu29xyvgvk5x9719cld2jdgmmqndt4" name="facebook-domain-verification">
<title>KB차차차 - 국내 최다 중고차 매물 등록대수 1위!</title>
<meta content="중고차 매매 사이트, 내차팔기, 중고차 경매, 자동차 시세, 중고차 판매, 수입차, 자동차 리스, 자동차 렌트, 인증중고차, 안심 중고차 플랫폼 KB차차차 공식 홈페이지" name="description"/>
<meta content="KB차차차 - 국내 최다 중고차 매물 등록대수 1위!" property="og:title">
<meta content="중고차 매매 사이트, 내차팔기, 중고차 경매, 자동차 시세, 중고차 판매, 수입차, 자동차 리스, 자동차 렌트, 인증중고차, 안심 중고차 플랫폼 KB차차차 공식 홈페이지" property="og:description">
<meta content="https://www.kbchachacha.com/images/common/og-thumb.png" property="og:image"/>
<meta content="website" property="og:type"/>
<meta content="ko_KR" property="og:locale"/>
<link href="/favicon.ico" refl="shortcut icon" type="image/x-icon"/>
<link href="/favicon.ico" rel="icon" type="image/x-icon"/>
<script language="javascript">
	var IMAGE_HOST = "https://img.kbchachacha.com";
</script>
<link href="

In [29]:
detail = 'https://www.kbchachacha.com/public/car/detail.kbc'
layer  = 'https://www.kbchachacha.com/public/layer/car/option/list.kbc'
master = 'https://www.kbchachacha.com/public/car/option/code/list.json'
car_seq = '27470105'

s = requests.Session()
base = {'User-Agent':'Mozilla/5.0','Accept-Language':'ko-KR,ko;q=0.9'}

# 1) 상세/레이어로 세션 쿠키 확보
s.get(detail, params={'carSeq':car_seq}, headers=base, timeout=15)
s.post(layer, data={'carSeq':car_seq}, headers={
    **base, 'X-Requested-With':'XMLHttpRequest',
    'Referer': f'{detail}?carSeq={car_seq}',
    'Content-Type':'application/x-www-form-urlencoded',
    'Accept':'text/html, */*;q=0.1',
}, timeout=15)

# 2) 마스터 AJAX POST
r = s.post(master, headers={
    **base,
    'Accept':'application/json, text/plain, */*',
    'X-Requested-With':'XMLHttpRequest',
    'x-ajax':'true',
    'Referer': layer,  # 브라우저 캡처와 동일
}, timeout=15)
r.raise_for_status()
print(r.status_code, r.headers.get('content-type'))

data = r.json()
# 3) 안전 파싱 (루트가 list or dict 모두 대응)
if isinstance(data, list):
    m = data
elif isinstance(data, dict) and 'optionList' in data:
    m = data['optionList']
else:
    raise RuntimeError(f'예상치 못한 형식: keys={list(data)[:10]}')

# 4) 매핑 딕셔너리
name_to_meta = {x['optionName'].strip(): x for x in m}
code_to_meta = {x['optionCode']: x for x in m}
print('옵션 마스터 수:', len(m))

data

200 application/json;charset=UTF-8
옵션 마스터 수: 69


{'optionList': [{'optionCode': '100130',
   'optionName': '썬루프(일반)',
   'optionShortName': '',
   'optionGbn': '039110',
   'optionGbnName': '외장/내장',
   'orderSeq': 0,
   'useYn': '',
   'fileName': '004.jpg',
   'optionRemark': '선루프는 자동차 지붕에 고강도 강화유리를 설치하여 지붕 일부 또는 전부를 개폐할 수 있는 장치 입니다. 선루프에 설치되는 강화유리는 대부분 자외선과 가시광선을 차단하며 효과적인 채광과 환기가 가능 합니다.',
   'modifyUser': '',
   'modifyDate': '',
   'regiUser': '',
   'regiDate': ''},
  {'optionCode': '200130',
   'optionName': '썬루프(파노라마)',
   'optionShortName': '',
   'optionGbn': '039110',
   'optionGbnName': '외장/내장',
   'orderSeq': 0,
   'useYn': '',
   'fileName': '012.jpg',
   'optionRemark': '파노라마선루프는 천장의 일부만 유리로 되어있는 선루프와 달리 천장의 대부분이 유리로 되어 있어 개방감을 더 늘리고 채광과 환기 성능을 더 높인 장치 입니다.',
   'modifyUser': '',
   'modifyDate': '',
   'regiUser': '',
   'regiDate': ''},
  {'optionCode': '200110',
   'optionName': '전동접이식 사이드 미러',
   'optionShortName': '',
   'optionGbn': '039110',
   'optionGbnName': '외장/내장',
   'orderSeq': 0,
   'useYn': '',
   'file

In [22]:
import requests
from bs4 import BeautifulSoup

DETAIL = 'https://www.kbchachacha.com/public/car/detail.kbc'
LAYER  = 'https://www.kbchachacha.com/public/layer/car/option/list.kbc'
MASTER = 'https://www.kbchachacha.com/public/car/option/code/list.json'
car_seq = '27470105'

s = requests.Session()
base = {'User-Agent':'Mozilla/5.0','Accept-Language':'ko-KR,ko;q=0.9'}

# 1) 세션 확보 + 레이어 HTML
s.get(DETAIL, params={'carSeq': car_seq}, headers=base, timeout=15)
r = s.post(LAYER, data={'carSeq': car_seq}, headers={
    **base,
    'X-Requested-With':'XMLHttpRequest',
    'Referer': f'{DETAIL}?carSeq={car_seq}',
    'Content-Type':'application/x-www-form-urlencoded',
    'Accept':'text/html, */*;q=0.1',
}, timeout=15)
soup = BeautifulSoup(r.text, 'html.parser')

# 2) hidden 값에서 옵션 코드 추출
hidden = soup.select_one('input#carOption')
codes = [c for c in (hidden['value'].split(',') if hidden else []) if c]

# 3) 옵션 마스터(POST) 불러오기
mr = s.post(MASTER, headers={
    **base,
    'Accept':'application/json, text/plain, */*',
    'X-Requested-With':'XMLHttpRequest',
    'x-ajax':'true',
    'Referer': LAYER,
}, timeout=15)
data = mr.json()
master = data['optionList'] if isinstance(data, dict) else data
code2 = {m['optionCode']: m for m in master}

# 4) 장착 옵션 매핑
equipped = [code2[c] for c in codes if c in code2]

# 5) 보기 좋게 정리
result = [
    {'code': o['optionCode'], 'name': o['optionName'], 'group': o['optionGbnName']}
    for o in equipped
]
for row in result:
    print(row)

{'code': '100100', 'name': '열선시트(앞좌석)', 'group': '시트'}
{'code': '100120', 'name': '주차감지센서(후방)', 'group': '안전'}
{'code': '100150', 'name': '스마트키', 'group': '편의/멀티'}
{'code': '200100', 'name': '알루미늄휠', 'group': '외장/내장'}
{'code': '200110', 'name': '전동접이식 사이드 미러', 'group': '외장/내장'}
{'code': '300110', 'name': '스티어링 휠 리모컨', 'group': '외장/내장'}
{'code': '300120', 'name': '가죽시트', 'group': '시트'}
{'code': '400100', 'name': '에어백(운전석)', 'group': '안전'}
{'code': '400110', 'name': '에어백(동승석)', 'group': '안전'}
{'code': '400150', 'name': '브레이크 잠김 방지(ABS)', 'group': '안전'}
{'code': '500120', 'name': '풀오토에어컨', 'group': '편의/멀티'}
{'code': '500180', 'name': 'ECM룸밀러', 'group': '외장/내장'}
{'code': '500190', 'name': '크루즈컨트롤(일반)', 'group': '편의/멀티'}
{'code': '500230', 'name': '레인센서와이퍼', 'group': '편의/멀티'}
{'code': '600100', 'name': 'CD플레이어', 'group': '편의/멀티'}
{'code': '600150', 'name': 'USB', 'group': '편의/멀티'}
{'code': '700320', 'name': '블루투스', 'group': '편의/멀티'}
{'code': '700390', 'name': '텔레스코프 스티어링 휠', 'group': '외장/내장

In [ ]:
DETAIL = 'https://www.kbchachacha.com/public/car/detail.kbc'
car_seq = '27470105'

s = requests.Session()
hdr = {'User-Agent':'Mozilla/5.0','Accept-Language':'ko-KR,ko;q=0.9'}
html = s.get(DETAIL, params={'carSeq':car_seq}, headers=hdr, timeout=15).text

# 1) 스크립트에서 var carHistorySeq = "숫자";
m = re.search(r'carHistorySeq\s*=\s*"(\d+)"', html)
car_history_seq = m.group(1) if m else None
print('carHistorySeq:', car_history_seq)

# 2) 실패 시 모든 <script> 텍스트에서 탐색(보강)
if not car_history_seq:
    soup = BeautifulSoup(html, 'hteml.parser')
    txt = '\n'.join(s.get_text() for s in soup.find_all('script'))
    m = re.search(r'carHistorySeq\s*=\s*"(\d+)"', txt)
    car_history_seq = m.group(1) if m else None
    print('fallback carHistorySeq:', car_history_seq)

carHistorySeq: 51106609


In [5]:
#제조사 정보를 가져와서 국산/수입 구분을 위한 매핑을 만들어주는 함수
def get_maker_info():
    """제조사 정보를 가져와서 국산/수입 구분을 위한 매핑을 생성합니다."""
    maker_url = "https://www.kbchachacha.com/public/search/carMaker.json?page=1&sort=-orderDate"
    
    try:
        response = requests.get(maker_url, timeout=10)
        if response.status_code == 200:
            data = response.json()
            maker_info = {}
            
            # 국산차 정보
            for maker in data.get('result', {}).get('국산', []):
                maker_info[maker['makerCode']] = {
                    'makerName': maker['makerName'],
                    'countryCode': maker['countryCode'],
                    'count': maker['count']
                }
            
            # 수입차 정보
            for maker in data.get('result', {}).get('수입', []):
                maker_info[maker['makerCode']] = {
                    'makerName': maker['makerName'],
                    'countryCode': maker['countryCode'],
                    'count': maker['count']
                }
            
            print(f"✅ 제조사 정보 수집 완료: 총 {len(maker_info)}개 제조사")
            return maker_info
        else:
            print(f"❌ 제조사 정보 수집 실패: HTTP {response.status_code}")
            return {}
    except Exception as e:
        print(f"❌ 제조사 정보 수집 오류: {e}")
        return {}

def get_car_info_via_api(car_seqs):
    """API를 통해 차량 기본 정보를 수집합니다."""
    api_url = "https://www.kbchachacha.com/public/car/common/recent/car/list.json"
    car_seq_str = ",".join(car_seqs)
    
    payload = {
        'gotoPage': 1,
        'pageSize': len(car_seqs),
        'carSeqVal': car_seq_str
    }
    
    try:
        session = requests.Session()
        session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36',
            'Accept': '*/*',
            'Accept-Language': 'ko,ko-KR;q=0.9,en-US;q=0.8,en;q=0.7',
            'Referer': 'https://www.kbchachacha.com/public/search/main.kbc',
            'Host': 'www.kbchachacha.com'
        })
        
        response = session.post(api_url, data=payload, timeout=10)
        
        if response.status_code == 200:
            data = response.json()
            return data.get('list', [])
        else:
            print(f"❌ API 오류: {response.status_code}")
            return []
    except Exception as e:
        print(f"❌ API 요청 실패: {e}")
        return []
def get_car_detail_from_html(car_seq):
    """HTML에서 상세 정보를 파싱합니다. (연료/변속기/차종/색상/이미지)"""
    detail_url = "https://www.kbchachacha.com/public/car/detail.kbc"
    
    try:
        session = requests.Session()
        session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
            'Referer': 'https://www.kbchachacha.com/public/search/main.kbc'
        })
        
        response = session.get(detail_url, params={'carSeq': car_seq}, timeout=15)
        
        if response.status_code != 200:
            print(f"❌ HTML 요청 실패: {response.status_code}")
            return {}
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 기본정보 표에서 필요한 정보만 추출
        kv = {}
        for tr in soup.select('.detail-info-table tbody tr'):
            tds = tr.select('th,td')
            for i in range(0, len(tds), 2):
                k = tds[i].get_text(strip=True)
                v = tds[i+1].get_text(strip=True) if i+1 < len(tds) else ''
                kv[k] = v
        
        # 이미지 URL 추출 (첫 번째 이미지만)
        image_url = None
        img_elements = soup.select('#btnCarPhotoView img[src]')
        if img_elements:
            image_url = img_elements[0]['src']
        
        # HTML에서 파싱해야 하는 필드만 추출
        return {
            'fuel': kv.get('연료', ''),
            'transmission': kv.get('변속기', ''),
            'class': kv.get('차종', ''),  # '차종' 키로 '중형' 값 추출
            'color': kv.get('색상', ''),
            'image_url': image_url,
        }
        
    except Exception as e:
        print(f"❌ HTML 파싱 오류 (carSeq: {car_seq}): {e}")
        return {}

def get_car_options_from_html(car_seq):
    """HTML에서 옵션 정보를 파싱합니다."""
    detail_url = "https://www.kbchachacha.com/public/car/detail.kbc"
    layer_url = "https://www.kbchachacha.com/public/layer/car/option/list.kbc"
    master_url = "https://www.kbchachacha.com/public/car/option/code/list.json"
    
    try:
        session = requests.Session()
        base_headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36',
            'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7'
        }
        
        # 1) 상세 페이지로 세션 확보
        session.get(detail_url, params={'carSeq': car_seq}, headers=base_headers, timeout=15)
        
        # 2) 옵션 레이어 POST
        layer_headers = {
            **base_headers,
            'X-Requested-With': 'XMLHttpRequest',
            'Referer': f'{detail_url}?carSeq={car_seq}',
            'Content-Type': 'application/x-www-form-urlencoded',
            'Accept': 'text/html, */*;q=0.1',
        }
        
        layer_response = session.post(layer_url, data={'carSeq': car_seq}, headers=layer_headers, timeout=15)
        
        if layer_response.status_code != 200:
            return []
        
        # 3) 옵션 코드 추출
        soup = BeautifulSoup(layer_response.text, 'html.parser')
        hidden = soup.select_one('input#carOption')
        codes = [c for c in (hidden['value'].split(',') if hidden else []) if c]
        
        # 4) 옵션 마스터 데이터 가져오기
        master_headers = {
            **base_headers,
            'Accept': 'application/json, text/plain, */*',
            'X-Requested-With': 'XMLHttpRequest',
            'x-ajax': 'true',
            'Referer': layer_url,
        }
        
        master_response = session.post(master_url, headers=master_headers, timeout=15)
        
        if master_response.status_code != 200:
            return []
        
        data = master_response.json()
        master = data['optionList'] if isinstance(data, dict) else data
        code_to_meta = {m['optionCode']: m for m in master}
        
        # 5) 장착 옵션 매핑
        equipped = [code_to_meta[c] for c in codes if c in code_to_meta]
        
        return [
            {
                'code': o['optionCode'],
                'name': o['optionName'],
                'group': o['optionGbnName']
            }
            for o in equipped
        ]
        
    except Exception as e:
        print(f"❌ 옵션 파싱 오류 (carSeq: {car_seq}): {e}")
        return []

def create_vehicle_record(api_data, html_data, options_data, maker_info):
    """DB 테이블 구조에 맞는 차량 레코드를 생성합니다."""
    
    # API 데이터에서 기본 정보 추출
    car_seq = api_data.get('carSeq', '')
    maker_code = api_data.get('makerCode', '')
    
    # 제조사 정보 매핑
    country_code = '알수없음'
    if maker_code in maker_info:
        country_code = maker_info[maker_code]['countryCode']
    
    # 연식 처리 (INT 타입, 연도만) - API에서 직접 사용
    model_year = api_data.get('yymm', '')
    if model_year and len(str(model_year)) == 4:
        model_year = int(model_year)
    else:
        model_year = None
    
    # 등록일 처리 (INT 타입, YYYYMMDD 형식) - API에서 직접 사용
    first_registration_date = api_data.get('regiDay', '')
    if first_registration_date and len(str(first_registration_date)) == 8:
        first_registration_date = int(first_registration_date)
    else:
        first_registration_date = None
    
    # 가격 정보 (만원 단위)
    price = api_data.get('sellAmt', 0)
    if price:
        price = price  # 만원 단위로 변환
    
    # HTML 데이터에서 추가 정보 추출 (API에 없는 정보만)
    html_fuel = html_data.get('fuel', '')
    html_transmission = html_data.get('transmission', '')
    html_color = html_data.get('color', '')
    html_image = html_data.get('image_url', '')
    
    # DB 테이블 구조에 맞는 레코드 생성 (이미지의 22개 컬럼만)
    record = {
        'VehicleId': None,  # DB에서 자동 생성
        'VehicleNo': api_data.get('carNo', ''),  # 차량번호판
        'CarSeq': car_seq,  # 내부식별자
        'Platform': 'kbchachacha',  # 플랫폼
        'Origin': country_code,  # 시장(국산/수입)
        'CarType': html_data.get('class', '기타'),  # 차종 (HTML에서 파싱)
        'Manufacturer': api_data.get('makerName', ''),  # 제조사
        'Model': api_data.get('className', ''),  # 모델명
        'Generation': api_data.get('carName', ''),  # 세대
        'Trim': api_data.get('gradeName', ''),  # 트림(등급)
        'FuelType': html_fuel or api_data.get('gasName', ''),  # 연료 (HTML 우선, API fallback)
        'Transmission': html_transmission or api_data.get('transmission', ''),  # 변속기 (HTML 우선, API fallback)
        'ColorName': html_color or api_data.get('color', ''),  # 차량색상 (HTML 우선, API fallback)
        'ModelYear': model_year,  # 차량연식 (INT, 연도만)
        'FirstRegistrationDate': first_registration_date,  # 차량등록일 (INT, YYYYMMDD)
        'Distance': api_data.get('km', 0),  # 주행거리
        'Price': price,  # 판매가 (만원 단위)
        'OriginPrice': None,  # 출시가 (API에서 제공되지 않음)
        'SellType': '일반',  # 판매형태
        'Location': api_data.get('cityName', ''),  # 차량 소재지
        'DetailURL': f"https://www.kbchachacha.com/public/car/detail.kbc?carSeq={car_seq}",  # 상세 페이지 URL
        'Photo': html_image,  # 차량 이미지 URL (HTML에서 파싱)
    }
    
    return record

def crawl_complete_car_info(car_seqs, delay=1):
    """완전한 차량 정보를 크롤링합니다."""
    
    print(f"🚀 완전한 차량 정보 크롤링 시작 (총 {len(car_seqs)}대)")
    
    # 1. 제조사 정보 수집
    print("📋 제조사 정보 수집 중...")
    maker_info = get_maker_info()
    
    # 2. API를 통한 기본 정보 수집
    print("🔍 API를 통한 기본 정보 수집 중...")
    api_data_list = get_car_info_via_api(car_seqs)
    
    if not api_data_list:
        print("❌ API 데이터 수집 실패")
        return []
    
    print(f"✅ API 데이터 수집 완료: {len(api_data_list)}대")
    
    # 3. 각 차량별로 HTML 상세 정보 수집
    complete_records = []
    
    for i, api_data in enumerate(api_data_list, 1):
        car_seq = api_data.get('carSeq', '')
        print(f"\n📄 {i}/{len(api_data_list)} - carSeq: {car_seq} 처리 중...")
        
        # HTML 상세 정보 수집
        html_data = get_car_detail_from_html(car_seq)
        
        # 옵션 정보 수집
        options_data = get_car_options_from_html(car_seq)
        
        # 완전한 레코드 생성
        record = create_vehicle_record(api_data, html_data, options_data, maker_info)
        complete_records.append(record)
        
        print(f"   ✅ 완료: {record['Manufacturer']} {record['Model']} {record['Generation']}")
        print(f"   💰 가격: {record['Price']}만원, 🚗 주행거리: {record['Distance']:,}km")
        print(f"   🎨 색상: {record['ColorName']}, ⚙️ 옵션: {len(options_data)}개")
        
        # 서버 부하 방지
        if i < len(api_data_list):
            time.sleep(delay)
    
    print(f"\n🎉 크롤링 완료! 총 {len(complete_records)}대의 완전한 정보 수집")
    return complete_records

# 테스트 실행
test_car_seqs = ['27490092', '27483488', '27471263']

print(f"{'='*80}")
print("🚀 DB 테이블 구조에 맞는 완전한 차량 정보 크롤링 시스템")
print(f"{'='*80}")

complete_records = crawl_complete_car_info(test_car_seqs, delay=2)

# 결과 출력
if complete_records:
    print(f"\n{'='*80}")
    print("📊 수집된 완전한 차량 정보")
    print(f"{'='*80}")
    
    for i, record in enumerate(complete_records, 1):
        print(f"\n{i}. {record['Manufacturer']} {record['Model']} {record['Generation']}")
        print(f"CarSeq: {record['CarSeq']}")
        print(f"차량번호: {record['VehicleNo']}")
        print(f"출신: {record['Origin']} ({record['CarType']})")
        print(f"가격: {record['Price']}만원")
        print(f"연식: {record['ModelYear']}")
        print(f"주행거리: {record['Distance']:,}km")
        print(f"연료: {record['FuelType']}")
        print(f"변속기: {record['Transmission']}")
        print(f"색상: {record['ColorName']}")
        print(f"지역: {record['Location']}")
        print(f"이미지: {record['Photo']}")
        print(f"상세URL: {record['DetailURL']}")
else:
    print("❌ 차량 정보 수집에 실패했습니다.")
    


🚀 DB 테이블 구조에 맞는 완전한 차량 정보 크롤링 시스템
🚀 완전한 차량 정보 크롤링 시작 (총 3대)
📋 제조사 정보 수집 중...
✅ 제조사 정보 수집 완료: 총 82개 제조사
🔍 API를 통한 기본 정보 수집 중...
✅ API 데이터 수집 완료: 3대

📄 1/3 - carSeq: 27490092 처리 중...
   ✅ 완료: 기아 레이 더 뉴 기아 레이
   💰 가격: 1630만원, 🚗 주행거리: 19,707km
   🎨 색상: 검정색, ⚙️ 옵션: 19개

📄 2/3 - carSeq: 27483488 처리 중...
   ✅ 완료: 기아 K9 더 뉴 K9(2세대)
   💰 가격: 4590만원, 🚗 주행거리: 20,641km
   🎨 색상: 검정색, ⚙️ 옵션: 57개

📄 3/3 - carSeq: 27471263 처리 중...
   ✅ 완료: 현대 쏘나타 쏘나타 디 엣지 하이브리드(DN8)
   💰 가격: 3210만원, 🚗 주행거리: 36,914km
   🎨 색상: 인기색상, ⚙️ 옵션: 45개

🎉 크롤링 완료! 총 3대의 완전한 정보 수집

📊 수집된 완전한 차량 정보

1. 기아 레이 더 뉴 기아 레이
CarSeq: 27490092
차량번호: 116소1868
출신: 국산 (경차)
가격: 1630만원
연식: 2023
주행거리: 19,707km
연료: 가솔린
변속기: 오토
색상: 검정색
지역: 경기
이미지: https://img.kbchachacha.com/IMG/carimg/l/img09/img2749/27490092_4656596854618635.jpeg
상세URL: https://www.kbchachacha.com/public/car/detail.kbc?carSeq=27490092

2. 기아 K9 더 뉴 K9(2세대)
CarSeq: 27483488
차량번호: 241모6906
출신: 국산 (대형)
가격: 4590만원
연식: 2023
주행거리: 20,641km
연료: 가솔린
변속기: 오토
색상: 검정색
지역: 인천
이미지: https://im

In [6]:
# 테스트 실행
test_car_seqs = ['27490092', '27483488', '27471263']

print(f"{'='*80}")
print("🚀 DB 테이블 구조에 맞는 완전한 차량 정보 크롤링 시스템")
print(f"{'='*80}")

complete_records = crawl_complete_car_info(test_car_seqs, delay=2)

# 데이터셋 확인
if complete_records:
    print(f"\n{'='*80}")
    print("📊 데이터셋 구조 확인")
    print(f"{'='*80}")
    
    # 첫 번째 레코드로 구조 확인
    first_record = complete_records[0]
    print(f"총 레코드 수: {len(complete_records)}")
    print(f"첫 번째 레코드의 필드 수: {len(first_record)}")
    print(f"\n📋 필드 목록:")
    for i, (key, value) in enumerate(first_record.items(), 1):
        print(f"   {i:2d}. {key}: {type(value).__name__} = {value}")
    
    # DataFrame으로 변환 테스트
    import pandas as pd
    df = pd.DataFrame(complete_records)
    print(f"\n�� DataFrame 정보:")
    print(f"   Shape: {df.shape}")
    print(f"   Columns: {list(df.columns)}")
    print(f"   Data types:")
    for col, dtype in df.dtypes.items():
        print(f"      {col}: {dtype}")
    
    # 각 레코드별 요약
    print(f"\n�� 각 차량별 요약:")
    for i, record in enumerate(complete_records, 1):
        print(f"   {i}. {record['Manufacturer']} {record['Model']} {record['Generation']}")
        print(f"      CarSeq: {record['CarSeq']}, 가격: {record['Price']}만원, 연식: {record['ModelYear']}")
        print(f"      차종: {record['CarType']}, 연료: {record['FuelType']}, 색상: {record['ColorName']}")
        
    # NULL 값 체크
    print(f"\n�� NULL 값 체크:")
    for col in df.columns:
        null_count = df[col].isnull().sum()
        if null_count > 0:
            print(f"   {col}: {null_count}개 NULL")
    
else:
    print("❌ 차량 정보 수집에 실패했습니다.")

🚀 DB 테이블 구조에 맞는 완전한 차량 정보 크롤링 시스템
🚀 완전한 차량 정보 크롤링 시작 (총 3대)
📋 제조사 정보 수집 중...
✅ 제조사 정보 수집 완료: 총 82개 제조사
🔍 API를 통한 기본 정보 수집 중...
✅ API 데이터 수집 완료: 3대

📄 1/3 - carSeq: 27490092 처리 중...
   ✅ 완료: 기아 레이 더 뉴 기아 레이
   💰 가격: 1630만원, 🚗 주행거리: 19,707km
   🎨 색상: 검정색, ⚙️ 옵션: 19개

📄 2/3 - carSeq: 27483488 처리 중...
   ✅ 완료: 기아 K9 더 뉴 K9(2세대)
   💰 가격: 4590만원, 🚗 주행거리: 20,641km
   🎨 색상: 검정색, ⚙️ 옵션: 57개

📄 3/3 - carSeq: 27471263 처리 중...
   ✅ 완료: 현대 쏘나타 쏘나타 디 엣지 하이브리드(DN8)
   💰 가격: 3210만원, 🚗 주행거리: 36,914km
   🎨 색상: 인기색상, ⚙️ 옵션: 45개

🎉 크롤링 완료! 총 3대의 완전한 정보 수집

📊 데이터셋 구조 확인
총 레코드 수: 3
첫 번째 레코드의 필드 수: 22

📋 필드 목록:
    1. VehicleId: NoneType = None
    2. VehicleNo: str = 116소1868
    3. CarSeq: int = 27490092
    4. Platform: str = kbchachacha
    5. Origin: str = 국산
    6. CarType: str = 경차
    7. Manufacturer: str = 기아
    8. Model: str = 레이
    9. Generation: str = 더 뉴 기아 레이
   10. Trim: str = 프레스티지
   11. FuelType: str = 가솔린
   12. Transmission: str = 오토
   13. ColorName: str = 검정색
   14. ModelYear: int = 2

In [ ]:
import sqlalchemy 



def save_kb_chchacha_DB():
  